In [1]:
import time
import numpy as np
import pandas as pd
import random
import torch as t
import copy
from fastcore.foundation import patch
from nixtla.data.ontsdataset import TimeSeriesDataset
from nixtla.data.ontsloader_fast import TimeSeriesLoader as TimeSeriesLoaderFast
# from nixtla.data.tsloader_pinche import TimeSeriesLoader as TimeSeriesLoaderPinche
# from nixtla.data.tsloader_general import TimeSeriesLoader as TimeSeriesLoaderGeneral

from nixtla.models.nbeats.onnbeats import Nbeats

## READ ON ELECTRICITY LOAD DATA

In [3]:
from nixtla.data.datasets.on import load_on_data

Y_insample_df, X_insample_df, Y_outsample_df, X_outsample_df, f_cols = load_on_data('2020-11-01')

ts_train_mask = np.ones(len(Y_insample_df))
ts_train_mask[-7*6*16:] = 0 # 16 fifteenminutales = 4 hours   (total = 1 week)
dataset = TimeSeriesDataset(Y_df=Y_insample_df, S_df=None, X_df=X_insample_df, ts_train_mask=ts_train_mask)
print('X: time series features, of shape (#series,#times,#features): \t' + str(X_insample_df.shape))
print('Y: target series (in X), of shape (#series,#times): \t \t' + str(Y_insample_df.shape))
# print('S: static features, of shape (#series,#features): \t \t' + str(S.shape))
Y_insample_df.head()

Processing dataframes ...
Creating ts tensor ...
X: time series features, of shape (#series,#times,#features): 	(104641, 331)
Y: target series (in X), of shape (#series,#times): 	 	(104641, 3)


,unique_id,ds,y
0,demanda_final,2017-10-07 00:00:00,5545.98338
1,demanda_final,2017-10-07 00:15:00,5448.66753
2,demanda_final,2017-10-07 00:30:00,5392.92989
3,demanda_final,2017-10-07 00:45:00,5371.67098
4,demanda_final,2017-10-07 01:00:00,5296.58436


## TRAIN AND VALIDATION DATALOADERS

In [4]:
train_loader = TimeSeriesLoaderFast(ts_dataset=dataset,
                                    model='nbeats',
                                    offset=0,
                                    window_sampling_limit=60*6*16, 
                                    input_size=3*16,
                                    output_size=16,
                                    idx_to_sample_freq=1,
                                    batch_size=256,
                                    is_train_loader=True)

val_loader = TimeSeriesLoaderFast(ts_dataset=dataset,
                                  model='nbeats',
                                  offset=0,
                                  window_sampling_limit=60*6*16,
                                  input_size=3*16,
                                  output_size=16,
                                  idx_to_sample_freq=1,
                                  batch_size=256,
                                  is_train_loader=False)

In [5]:
nbeatsx = Nbeats(input_size_multiplier=7,
                 output_size=24,
                 shared_weights=False,
                 stack_types=['exogenous_g_a']+3*['identity'],
                 n_blocks=4*[1],
                 n_layers=4*[2],
                 n_hidden=4*[[256,256]],
                 n_harmonics=1,
                 n_polynomials=2,
                 x_s_n_hidden=0,
                 exogenous_n_channels=9,
                 f_cols=f_cols,
                 batch_normalization=False,
                 dropout_prob=0.1,
                 theta_with_exogenous=True,
                 learning_rate=0.001,
                 lr_decay=0.5,
                 n_lr_decay_steps=3,
                 weight_decay=0.0000001,
                 l1_lambda_x=0.0001,
                 n_iterations=100,
                 early_stopping=10,
                 loss='MAE',
                 frequency=24,
                 random_seed=1,
                 seasonality='H')

nbeatsx.fit(train_ts_loader=train_loader, val_ts_loader=val_loader, verbose=True, eval_steps=5)

TypeError: argument of type 'NoneType' is not iterable